In [1]:
import math

def dataset_split(data, arc, val):
    newData = []
    for rec in data:
        if rec[arc] == val:
            reducedSet = list(rec[:arc])
            reducedSet.extend(rec[arc+1:])
            newData.append(reducedSet)
    return newData

In [2]:
def entropy_calc(data):
    entries = len(data)
    labels = {}
    
    for rec in data:
        label = rec[-1]
        if label not in labels.keys():
            labels[label] = 0
        labels[label] += 1
    entropy = 0.0
    for key in labels:
        prob = float(labels[key])/entries
        entropy -= prob * math.log(prob,2)
    return entropy            

In [3]:
def attribute_selection(data):
    features = len(data[0])-1
    baseEntropy = entropy_calc(data)
    max_info_gain = 0.0
    best_attr = -1
    
    for i in range(features):
        attr_list = [rec[i] for rec in data]
        unique_vals = set(attr_list)
        newEntropy = 0.0
        attrEntropy = 0.0
        
        for value in unique_vals:
            new_data = dataset_split(data, i, value)
            prob = float(len(new_data))/len(data)
            newEntropy = prob * entropy_calc(new_data)
            attrEntropy += newEntropy
        infoGain = baseEntropy - attrEntropy
        if infoGain>max_info_gain:
            max_info_gain = infoGain
            best_attr = i
    return best_attr

In [4]:
def decision_tree(data, labels):
    classList = [rec[-1] for rec in data]

    if classList.count(classList[0]) == len(classList):
        return classList[0]
    
    maxInfoNode = attribute_selection(data)
    treeLabels = labels[maxInfoNode]
    
    theTree = {treeLabels : {}}
    del(labels[maxInfoNode])
    
    node_values = [rec[maxInfoNode] for rec in data]
    unique_vals = set(node_values)
    
    for value in unique_vals:
        sublabel = labels[:]
        theTree[treeLabels][value] = decision_tree(dataset_split(data, maxInfoNode, value), sublabel)
    return theTree

In [5]:
def print_tree(tree, level):
    if tree=='yes' or tree=='no':
        print(''*level, 'd=', tree)
        return
    for key, value in tree.items():
        print(''*level, key)
        print_tree(value, level*2)

In [6]:
with open('data/tennis.csv') as file:
    fdata = [line.strip() for line in file]
    meta_data = fdata[0].split(',')
    train_data = [x.split(',') for x in fdata[1:]]

tree = decision_tree(train_data, meta_data)
print(tree)

{'outlook': {'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}, 'overcast': 'yes', 'rain': {'wind': {'strong': 'no', 'weak': 'yes'}}}}


In [7]:
print_tree(tree, 1)

 outlook
 sunny
 humidity
 high
 d= no
 normal
 d= yes
 overcast
 d= yes
 rain
 wind
 strong
 d= no
 weak
 d= yes
